In [2]:
import org.apache.spark.sql.{SQLContext, SaveMode}
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

sqlContext = org.apache.spark.sql.SQLContext@1d31c97b


Some(http://iuni2:4040)

In [3]:
//val sqlContext = new SQLContext(sc)
val df = sqlContext.read
  .format("com.databricks.spark.xml")
  .option("rowTag", "REC")
  //.schema(customSchema)
  .load("/dataRCSC/*.xml")

df.show(2)
df.printSchema()

+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+--------+---------+-----------------+--------------------+
|                UID|    _r_id_disclaimer|abstracts|addresses|category_info|citedWork|contributors|        dynamic_data|fund_ack|item|keywords|reference|reprint_addresses|         static_data|
+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+--------+---------+-----------------+--------------------+
|WOS:000531777000006|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|    null|     null|             null|[,,,,, [, [, 2, [...|
|WOS:000244816400003|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|    null|     null|             null|[,,,, [2, [[[, F-...|
+-------------------+--------------

df = [UID: string, _r_id_disclaimer: string ... 12 more fields]


[UID: string, _r_id_disclaimer: string ... 12 more fields]

In [4]:
val WoS2 = df.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name", "static_data.fullrecord_metadata.reprint_addresses",
                      "static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.keywords.keyword","static_data.fullrecord_metadata.references.reference",
                      "static_data.contributors.contributor","static_data.summary.EWUID",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 |    |    |-- suffix: string (nullable = true)
 | 

WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 14 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 14 more fields]

In [7]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

//WoS2.write.mode(SaveMode.Overwrite).json("/dataRCSC/WoS_json")
val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").orderBy($"cited".desc).coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/dataRCSC/WoS_json/citeEdges.csv/")

WoSref = [citing: string, refList: array<string> ... 1 more field]


[citing: string, refList: array<string> ... 1 more field]

In [6]:
val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/dataRCSC/WoS_json/refEdges.csv")

WoSref = [citing: string, refList: array<string> ... 1 more field]


[citing: string, refList: array<string> ... 1 more field]

In [2]:
val WoS = spark.read.format("parquet").load("/WoSraw/parquet/*.parquet")

val WoS2 = WoS.select("UID","static_data.summary.titles","static_data.fullrecord_metadata.category_info")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- category_info: struct (nullable = true)
 |    |-- headings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- heading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subheadings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subheading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subjects: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subject: array (nullable = true)
 |    |    |    |-- element: struct (con

WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 1 more field]


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 1 more field]

In [8]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)
val sdf = spark.read.format("csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("escape", "\"")
    //.option("inferSchema","true")
    //.load("/dataRCSC/metaInstitutes.csv")
    .load("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/metaInstitutes-08-03.csv")
sdf.describe().filter($"summary" === "count").show

+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+
|summary|   _c0|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation|
+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+
|  count|207311|  207311|87409|  207311|207255|146317|91304|   123955| 207311|  147637|      207294| 200607| 195418|     0|           57491|    2759|         87409|         66516|160560|178607|     66516|         66516|
+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+----

session = org.apache.spark.sql.SparkSession@43fa97a0
azureFile = /azureBlob.txt MapPartitionsRDD[50] at textFile at <console>:75
sdf = [_c0: string, cord_uid: string ... 20 more fields]


lastException: Throwable = null


[_c0: string, cord_uid: string ... 20 more fields]

In [3]:
//sdf.printSchema
sdf.filter("sha is not null").show(false)

+--------+----------------------------------------+--------+--------------------------------------------------------------------------------------------------------------------------------+------------------------+---------+---------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
val magPapers = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/mag/Papers.txt")
magPapers.show

+-----+-----+--------------------+------+-----------------------------------+-----------------------------------+----+----+----------+----------------------------+----+----+----+----+----+----+----+----+----+----+----------------------------------+----+-------------------+
|  _c0|  _c1|                 _c2|   _c3|                                _c4|                                _c5| _c6| _c7|       _c8|                         _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|                              _c20|_c21|               _c22|
+-----+-----+--------------------+------+-----------------------------------+-----------------------------------+----+----+----------+----------------------------+----+----+----+----+----+----+----+----+----+----+----------------------------------+----+-------------------+
| 8933|30118|                null|  null|               историческая наук...|               Историческая наук...|null|1960|1960-10-31|        ТОО Редакция журн...|null|null|null|

magPapers = [_c0: bigint, _c1: int ... 21 more fields]


lastException: Throwable = null


[_c0: bigint, _c1: int ... 21 more fields]

In [41]:
val doiMatch = sdf.join(magPapers, sdf("doi")===magPapers("_c2"), "left").cache()
doiMatch.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|  url| s2_id|  _c0|  _c1|  _c2|  _c3|  _c4|  _c5|_c6|  _c7|  _c8|  _c9| _c10|_c11|_c12|_c13|_c14| _c15| _c16| _c17| _c18| _c19| _c20|_c21|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+
|  count|  128648|55847|  128648|128620|100773|60927|    99169| 128

doiMatch = [cord_uid: string, sha: string ... 40 more fields]


[cord_uid: string, sha: string ... 40 more fields]

In [9]:
val idMap = spark.read.format("csv").option("header", false).load("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/2020-08-04-CORD-UID-MappedTo-2020-07-29-MAG-ID.csv")
val idMatch = sdf.join(idMap, sdf("cord_uid")===idMap("_c0"), "left").withColumnRenamed("_c1","MAGid").drop("_c0")
idMatch.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation| MAGid|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+
|  count|  209004|88204|  209004|208946|147939|92205|   125284| 209004|  148791|      208986| 202268| 197107|     0|           57974|    2764|         88204|         67127|162199|180270|     67127|         67127|184143|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----

idMap = [_c0: string, _c1: string]
idMatch = [cord_uid: string, sha: string ... 20 more fields]


[cord_uid: string, sha: string ... 20 more fields]

In [10]:
val idMatch2 = idMatch.groupBy("cord_uid").agg(concat_ws(";", collect_list("MAGid")) as "MAGids")
idMatch2.describe().filter($"summary" === "count").show

+-------+--------+------+
|summary|cord_uid|MAGids|
+-------+--------+------+
|  count|  205829|205829|
+-------+--------+------+



idMatch2 = [cord_uid: string, MAGids: string]


[cord_uid: string, MAGids: string]

In [5]:
val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
magMatch.describe().filter($"summary" === "count").show

Name: Compile Error
Message: <console>:44: error: not found: value magPapers
       val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
                                   ^
<console>:44: error: not found: value magPapers
       val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
                                                                 ^

StackTrace: 

In [11]:
val magPPA = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/mag/PaperAuthorAffiliations.txt")

magPPA.show

+---+----------+----------+---+--------------------+--------------------+
|_c0|       _c1|       _c2|_c3|                 _c4|                 _c5|
+---+----------+----------+---+--------------------+--------------------+
|  9|2632942543|      null|  1|Victoriano Perruc...|                null|
| 15| 199142497|      null|  1|     Robert Münscher|Heidelberg, Deuts...|
| 15| 680395887|      null|  2|       Julia Hormuth|Reutlingen, Deuts...|
| 23|1243978490|  79576946|  1|     Eric T Stoopler|Department of Ora...|
| 23|2582258949|      null|  3| Arthur S Kuperstein|                null|
| 23|2582736345|      null|  2|        Ying Wai Sia|                null|
| 58|2641488431|      null|  1|             I. Uray|                null|
| 58|2693978806|      null|  2|           P. Kovacs|                null|
| 79|2662843304|      null|  1|           K. Suresh|                null|
| 79|2683440697|      null|  2|     T. S. J. Norman|                null|
| 87|2676159639|      null|  1| Gabrie

magPPA = [_c0: bigint, _c1: bigint ... 4 more fields]


[_c0: bigint, _c1: bigint ... 4 more fields]

In [12]:
val magPPA2 = magPPA.toDF("MAGid2","authorID","affiliationID","authorOrder","author","affiliation").filter("affiliationID is not null")
val magMatchPPA = idMatch.join(magPPA2, idMatch("MAGid")===magPPA2("MAGid2"), "left").cache()
magMatchPPA.describe().filter($"summary" === "count").show

+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+------+--------+-------------+-----------+------+-----------+
|summary|cord_uid|   sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation| MAGid|MAGid2|authorID|affiliationID|authorOrder|author|affiliation|
+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+------+--------+-------------+-----------+------+-----------+
|  count|  795656|400653|  795656|795585|642212|384412|   556546| 795656|  647090|      795607| 781626| 749525|     0|          160541|    5515|       

magPPA2 = [MAGid2: bigint, authorID: bigint ... 4 more fields]
magMatchPPA = [cord_uid: string, sha: string ... 26 more fields]


[cord_uid: string, sha: string ... 26 more fields]

In [14]:
val magAffiliation = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/mag/Affiliations.txt")

magAffiliation.show

+--------+-----+--------------------+--------------------+-------------+--------------------+--------------------+-----+-----+-------+----------+----------+-------------------+
|     _c0|  _c1|                 _c2|                 _c3|          _c4|                 _c5|                 _c6|  _c7|  _c8|    _c9|      _c10|      _c11|               _c12|
+--------+-----+--------------------+--------------------+-------------+--------------------+--------------------+-----+-----+-------+----------+----------+-------------------+
|20455151| 9939|         air liquide|         Air Liquide|grid.476009.c|https://web.archi...|http://en.wikiped...| 6667| 6659|  55418|52.5035934|  -1.80516|2016-06-24 00:00:00|
|24386293|13887|hellenic national...|Hellenic National...|         null|http://www.hnms.g...|http://en.wikiped...|   84|   84|   1871|  37.97614|   23.7364|2016-06-24 00:00:00|
|32956416|12964|catholic universi...|Catholic Universi...|grid.448708.7|  http://www.uco.fr/|http://en.wikiped...| 

magAffiliation = [_c0: bigint, _c1: int ... 11 more fields]


lastException: Throwable = null


[_c0: bigint, _c1: int ... 11 more fields]

In [15]:
val magAffiliation2 = magAffiliation.select("_c0","_c2","_c3","_c4","_c6","_c7","_c9","_c10","_c11").toDF("affiliationID2","normalizedName","displayName","GRIDid","wiki","paperTotal", "citationTotal", "Latitude", "Longitude")
val magMatchPPA2 = magMatchPPA.join(magAffiliation2, magMatchPPA("affiliationID")===magAffiliation2("affiliationID2"), "left").cache()
magMatchPPA2.describe().filter($"summary" === "count").show

+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+------+--------+-------------+-----------+------+-----------+--------------+--------------+-----------+------+------+----------+-------------+--------+---------+
|summary|cord_uid|   sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation| MAGid|MAGid2|authorID|affiliationID|authorOrder|author|affiliation|affiliationID2|normalizedName|displayName|GRIDid|  wiki|paperTotal|citationTotal|Latitude|Longitude|
+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+------+------

magAffiliation2 = [affiliationID2: bigint, normalizedName: string ... 7 more fields]
magMatchPPA2 = [cord_uid: string, sha: string ... 35 more fields]


[cord_uid: string, sha: string ... 35 more fields]

In [16]:
val PaperMAG = magMatchPPA2.groupBy("cord_uid").agg(concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    concat_ws(";", collect_list("authorID")) as "authorids",
                                                    concat_ws(";", collect_list("authorOrder")) as "authorOrders",
                                                    concat_ws(";", collect_list("affiliationID2")) as "affiliationids",
                                                    concat_ws(";", collect_list("normalizedName")) as "affiliationNormalized",
                                                    concat_ws(";", collect_list("displayName")) as "affiliationNames",
                                                    concat_ws(";", collect_list("GRIDid")) as "GRIDids")
PaperMAG.show(false)

+--------+-------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+--------------------+-------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------

PaperMAG = [cord_uid: string, MAGids: string ... 6 more fields]


[cord_uid: string, MAGids: string ... 6 more fields]

In [17]:
val PaperMAG2 = sdf.join(PaperMAG, Seq("cord_uid"), "left").drop("_c0").cache()
PaperMAG2.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+---------+------------+--------------+---------------------+----------------+-------+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation|MAGids|authorids|authorOrders|affiliationids|affiliationNormalized|affiliationNames|GRIDids|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+------+---------+------------+--------------+---------------------+----------------+-------+
|  count|  207311|87409|  207311|207255|146317|91304|   123955| 207311|  147637|     

PaperMAG2 = [cord_uid: string, sha: string ... 26 more fields]


[cord_uid: string, sha: string ... 26 more fields]

In [20]:
PaperMAG2.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/2020-08-03/CORD-19-MAGinstsFull.csv")

In [19]:
val insititutionMAG = magMatchPPA2.groupBy("affiliationID").agg(//concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    //concat_ws(";", collect_list("cord_uid")) as "cord_uids",
                                                    first("displayName") as "displayName",
                                                    first("normalizedName") as "normalizedName",
                                                    first("wiki") as "wiki",
                                                    first("paperTotal") as "paperTotal", 
                                                    first("citationTotal") as "citationTotal",
                                                    first("Latitude") as "Latitude",
                                                    first("Longitude") as "Longitude",
                                                    first("GRIDid") as "GRIDid")
insititutionMAG.show(false)

+-------------+------------------------------------------+------------------------------------------+-----------------------------------------------------------------------+----------+-------------+-----------+-----------+-------------+
|affiliationID|displayName                               |normalizedName                            |wiki                                                                   |paperTotal|citationTotal|Latitude   |Longitude  |GRIDid       |
+-------------+------------------------------------------+------------------------------------------+-----------------------------------------------------------------------+----------+-------------+-----------+-----------+-------------+
|746986       |Halmstad University                       |halmstad university                       |http://en.wikipedia.org/wiki/Halmstad_University,_Sweden               |2529      |38650        |56.66417   |12.87861   |grid.73638.39|
|6141010      |European Institute of Oncology       

insititutionMAG = [affiliationID: bigint, displayName: string ... 7 more fields]


[affiliationID: bigint, displayName: string ... 7 more fields]

In [21]:
insititutionMAG//.select("wosid","headings","subheadings","WC","SC","disciplines","subdisciplines","fractions")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/2020-08-03/MAGInstTable.csv")

In [7]:
val meta0 = spark.read.format("csv")
      .option("header", "true")
      //.option("delimiter","\t")
      .option("inferSchema","true")
      .load("/dataRCSC/metaAuthors.csv")

meta0.describe().filter($"summary" === "count").show

+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+
|summary|   _c0|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pdfAuthors|SSAuthors|pmcAffiliation|pdfAffiliation|SSRefs|
+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+
|  count|137411|  137411|63458|  137411|137370|113076|70527|   100629| 137359|  105932|      137366| 132106| 130231|  2649|           27941|    3719|         64965|         52370|124586|111762|     52315|     61

meta0 = [_c0: int, cord_uid: string ... 24 more fields]


[_c0: int, cord_uid: string ... 24 more fields]

In [3]:
val SS = spark.read.json("/dataRCSC/SS-2020-05-27/*.gz")
SS.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- doiUrl: string (nullable = true)
 |-- entities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fieldsOfStudy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- inCitations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journalName: string (nullable = true)
 |-- journalPages: string (nullable = true)
 |-- journalVolume: string (nullable = true)
 |-- magId: string (nullable = true)
 |-- outCitations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- paperAbstract: string (nullable = true)
 |-- pdfUrls: array (nullable = true)
 |    |-- element: string (containsNull = 

SS = [authors: array<struct<ids:array<string>,name:string>>, doi: string ... 19 more fields]


[authors: array<struct<ids:array<string>,name:string>>, doi: string ... 19 more fields]

In [21]:
val SS0 = SS.select("authors.name", "authors.ids", "doi", "id", "inCitations")
            .toDF("SSauthorNames","SSauthorIDs","SSdoi","SSid","SSrefs").cache()
val meta1 = meta0.join(SS0, meta0("sha")===SS0("SSid"), "left")
meta1.describe().filter($"summary" === "count").show

+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+-----+-----+
|summary|   _c0|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pdfAuthors|SSAuthors|pmcAffiliation|pdfAffiliation|SSRefs|SSdoi| SSid|
+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+-----+-----+
|  count|137411|  137411|63458|  137411|137370|113076|70527|   100629| 137359|  105932|      137366| 132106| 130231|  2649|           27941|    3719|         64965|         52

SS0 = [SSauthorNames: array<string>, SSauthorIDs: array<array<string>> ... 3 more fields]
meta1 = [_c0: int, cord_uid: string ... 29 more fields]


lastException: Throwable = null


[_c0: int, cord_uid: string ... 29 more fields]

In [51]:
val SSexp = SS0.select(explode($"SSauthorIDs"),size($"SSauthorNames")).withColumn("size", size($"col"))
SSexp.orderBy($"size".desc).show

+-----------+-------------------+----+
|        col|size(SSauthorNames)|size|
+-----------+-------------------+----+
| [51978410]|                  1|   1|
| [71630645]|                  3|   1|
| [52393267]|                  3|   1|
| [72050485]|                  3|   1|
|  [1776025]|                  6|   1|
|  [2836018]|                  6|   1|
| [47422037]|                  6|   1|
|  [1679387]|                  6|   1|
|[145799243]|                  6|   1|
|[145754722]|                  6|   1|
|[114402012]|                  1|   1|
|[136282636]|                  1|   1|
|[146709087]|                  3|   1|
| [51218054]|                  3|   1|
|[144681826]|                  3|   1|
|[120457898]|                  1|   1|
|[152634023]|                  4|   1|
|[153204165]|                  4|   1|
|  [4244638]|                  4|   1|
|  [1796371]|                  4|   1|
+-----------+-------------------+----+
only showing top 20 rows



SSexp = [col: array<string>, size(SSauthorNames): int ... 1 more field]


[col: array<string>, size(SSauthorNames): int ... 1 more field]

In [26]:
val meta2 = meta0.join(SS0, meta0("doi")===SS0("SSdoi"), "left").cache()
meta2.describe().filter($"summary" === "count").show

+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+-----+-----+
|summary|   _c0|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pdfAuthors|SSAuthors|pmcAffiliation|pdfAffiliation|SSRefs|SSdoi| SSid|
+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+-----+-----+
|  count|140652|  140652|65282|  140652|140611|116317|73156|   102945| 140600|  107198|      140607| 133820| 133425|  2682|           28189|    3751|         66806|         53

meta2 = [_c0: int, cord_uid: string ... 29 more fields]


[_c0: int, cord_uid: string ... 29 more fields]

In [29]:
meta2.printSchema

root
 |-- _c0: integer (nullable = true)
 |-- cord_uid: string (nullable = true)
 |-- sha: string (nullable = true)
 |-- source_x: string (nullable = true)
 |-- title: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- pmcid: string (nullable = true)
 |-- pubmed_id: string (nullable = true)
 |-- license: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- mag_id: string (nullable = true)
 |-- who_covidence_id: string (nullable = true)
 |-- arxiv_id: string (nullable = true)
 |-- pdf_json_files: string (nullable = true)
 |-- pmc_json_files: string (nullable = true)
 |-- url: string (nullable = true)
 |-- s2_id: string (nullable = true)
 |-- pmcAuthors: string (nullable = true)
 |-- pdfAuthors: string (nullable = true)
 |-- SSAuthors: string (nullable = true)
 |-- pmcAffiliation: string (nullable = true)
 |-- pdfAffiliation: string (nul

In [27]:
meta2.select("SSauthorNames","SSauthorIDs","SSrefs").show(false)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [31]:
val SSmatched = meta2.groupBy("cord_uid").agg(concat_ws(";", collect_list("SSdoi")) as "SSdois",
                                            first("SSauthorNames") as "SSauthorNames",
                                            first("SSauthorIDs") as "SSauthorIDs",
                                            first("SSrefs") as "SSrefs")

SSmatched.describe().filter($"summary" === "count").show

+-------+--------+------+
|summary|cord_uid|SSdois|
+-------+--------+------+
|  count|  136783|136783|
+-------+--------+------+



SSmatched = [cord_uid: string, SSdois: string ... 3 more fields]


[cord_uid: string, SSdois: string ... 3 more fields]

In [57]:
val SSmatched2 = SSmatched.withColumn("SSauthorNameList", array_join($"SSauthorNames", ";"))
                        .withColumn("SSauthorIDList", array_join(flatten($"SSauthorIDs"), ";"))
                        .withColumn("SSrefList", array_join($"SSrefs", ";"))
                        .select("cord_uid","SSdois","SSauthorNameList","SSauthorIDList","SSrefList")
SSmatched2.describe().filter($"summary" === "count").show

+-------+--------+------+----------------+--------------+---------+
|summary|cord_uid|SSdois|SSauthorNameList|SSauthorIDList|SSrefList|
+-------+--------+------+----------------+--------------+---------+
|  count|  136783|136783|           80453|         80453|    80453|
+-------+--------+------+----------------+--------------+---------+



SSmatched2 = [cord_uid: string, SSdois: string ... 3 more fields]


[cord_uid: string, SSdois: string ... 3 more fields]

In [56]:
val meta2 = meta0.join(SSmatched2, Seq("cord_uid"), "left").cache()
meta2.describe().filter($"summary" === "count").show

+-------+--------+------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+------+----------------+--------------+---------+
|summary|cord_uid|   _c0|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pdfAuthors|SSAuthors|pmcAffiliation|pdfAffiliation|SSRefs|SSdois|SSauthorNameList|SSauthorIDList|SSrefList|
+-------+--------+------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+------+----------------+--------------+---------+
|  count|  137411|137411|63458|  137411|137370|113076|70527|   1

meta2 = [cord_uid: string, _c0: int ... 28 more fields]


[cord_uid: string, _c0: int ... 28 more fields]

In [58]:
meta2.drop("_c0","SSAuthors","SSRefs")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/metaAuthors.csv")

In [68]:
val url = "https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb"
val result = scala.io.Source.fromURL(url).mkString
println(result)

{"abstract":"ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) w

url = https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb
result = 


"{"abstract":"ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred yea...
